In [0]:
bronze_df=spark.readStream.format("delta").load("dbfs:/Brjan2/01/24/20/newyear")

In [0]:
from pyspark.sql.functions import explode
from pyspark.sql import functions as f
from pyspark.sql.functions import col,lit
from datetime import datetime
# Importing sql types
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, DoubleType, FloatType, DateType
from pyspark.sql.functions  import date_format
from pyspark.sql.functions import current_timestamp

In [0]:
bronze_df1=bronze_df.withColumn("results", explode("results"))\
            .withColumn("Date", date_format(col("curr_timestamp"), "yyyy-MM-dd"))\
            .withColumn("Time", date_format(col("curr_timestamp"), "HH:mm:ss"))\
            .withColumn("Silver_timestamp",current_timestamp())\
            .select("nationality","results.user.gender","results.user.name.first","results.user.name.last","results.user.name.title","results.user.location.city","results.user.location.state","results.user.location.street","results.user.location.zip","results.user.email","results.user.picture.large","results.user.picture.medium","results.user.picture.thumbnail","results.user.password","results.user.dob","results.user.phone","results.user.cell", "results.user.registered","results.user.salt","results.user.sha256","results.user.username","results.user.sha1","results.user.md5","seed","version","curr_timestamp","Date","Time","Silver_timestamp")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,ArrayType
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
bronze_df2=bronze_df1.withColumn('username', regexp_replace('username', '[^a-zA-Z]', ''))\
    .withColumnRenamed('username',"nameoftheuser")

In [0]:
dataframe2= spark.read.format("org.apache.spark.sql.avro.AvroFileFormat").load("dbfs:/FileStore/avro/projectsample.avro")

In [0]:
from pyspark.sql.functions import broadcast

In [0]:
bronze_df3=bronze_df2.join(broadcast(dataframe2),bronze_df2["nameoftheuser"]==dataframe2["username"],"inner")

In [0]:
%sql
create table silver_external1_jan201200newyear
using delta
location "dbfs:/silverjan2/01/20/new/year"

In [0]:
silver_df=bronze_df3.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "checkpointLocationjan201202024year") \
    .option("mergeSchema",True) \
     .start("dbfs:/silverjan2/01/20/new/year")